In [1]:
import sys
import pickle
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os
from torch_geometric.data import Dataset, download_url, Data
import pandas as pd
import shutil
import torch
from scipy.spatial import distance

from torch_geometric import nn as gnn
import torch.nn.functional as F
from torch import nn

from torch_geometric.loader import NeighborLoader
from torch_geometric.sampler import BaseSampler

from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling

In [4]:
with open('C://Austin//Projects//MS_Thesis_Data//test_eye_clusters_extract.pk1', 'rb') as handle:
    data = pickle.load(handle)

In [3]:
data.keys()

dict_keys(['name', 'image', 'features', 'factor', 'cluster', 'centroids'])

In [31]:
class Separated_Dataset(Dataset): #For this ds I have ald done all the required pre prcessing 
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None, data_dict = None, num_data = 10, num_nodes = 5):
        self.data_dict = data_dict
        self.num_data = num_data
        self.num_nodes = num_nodes
        super().__init__(root, transform, pre_transform, pre_filter)
        
    @property
    def raw_file_names(self):
        return np.array([os.path.join(self.raw_dir, x) for x in self.data_dict["name"]])

    @property
    def processed_file_names(self):
        return "not_implemented.pt"
    
    def process(self):
        
        idx = -1

        # Process pre made data dictionary

        x_features = np.append(self.data_dict["features"], self.data_dict["centroids"], axis=0)
        y = np.append(self.data_dict["cluster"], np.array([127,128,129,130,131]), axis=0)
        
        edge_map = []


        i=-1
        for cluster in y[:127]:
            i+=1
            edge_map.append([np.int64(i), np.int64(cluster+self.data_dict["features"].shape[0])])
            #edge_map.append([np.int64(cluster+self.data_dict["features"].shape[0]), np.int64(i)])
        

        i+=1
        for centroid1 in y[i:-1]:
            for centroid2 in y[i+1:]:
                edge_map.append([np.int64(centroid1+self.data_dict["features"].shape[0]), np.int64(centroid2+self.data_dict["features"].shape[0])])
                #edge_map.append([np.int64(centroid2+self.data_dict["features"].shape[0]), np.int64(centroid1+self.data_dict["features"].shape[0])])
            i+=1

        edge_attrs = [] 


        for edge in edge_map:
            edge_attrs.append(1)
        
        edge_attrs = np.array(edge_attrs)
        edge_attrs = np.delete(edge_attrs,1)
        edge_attrs = torch.Tensor(edge_attrs)

        edge_map_aux = [None,None]
        edge_map_aux[0] = [x[0] for x in edge_map]
        edge_map_aux[1] = [x[1] for x in edge_map]
        edge_map_aux = np.array(edge_map_aux)

        for i in range(self.data_dict["image"].shape[0]):
            
            y = torch.Tensor(self.data_dict["image"][i])

            x_temp = np.delete(x_features,i, axis=0)
            edge_map_aux_temp = np.delete(edge_map_aux,i, axis = 1)


            data = Data(torch.Tensor(x_temp), torch.Tensor(edge_map_aux_temp), edge_attrs, y)

            torch.save(data, os.path.join(self.processed_dir, f'data_{i}.pt'))

    def len(self):
        return self.data_dict["name"].shape[0]

    def get(self, idx):
        data = torch.load(os.path.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [30]:
x = np.random.rand(127,4096)
print(x.shape)
print(np.delete(x,1, axis = 0).shape)
print(x.shape)

(127, 4096)
(126, 4096)
(127, 4096)


In [6]:
target_cluster = data["cluster"][0]
same_clust = np.where(data["cluster"].astype(np.int32)==np.int32(target_cluster))[0]
diff_clust = np.where(data["cluster"].astype(np.int32)!=np.int32(target_cluster))[0]
same_clust = np.random.choice(same_clust, 7, replace=False)
diff_clust = np.random.choice(diff_clust, 7, replace=False)

clusts_in_diff = data["cluster"][diff_clust]
print(np.unique(clusts_in_diff).shape)
x = data["features"][np.append(same_clust,diff_clust)]
x = np.append(x, data["centroids"][np.append(target_cluster, np.unique(clusts_in_diff))], axis=0)
print(x.shape)


(3,)
(18, 4096)


In [99]:
x[:14].shape

(14, 4096)

In [32]:
root_path = "C://Austin//Projects//MS_Thesis_Data//base_gnn_testing_root//separated_training_set"
dataset = Separated_Dataset(root_path, data_dict=data)

Processing...
Done!


Separated_Dataset(127)

In [8]:
dataset = torch.load("C://Austin//Projects//MS_Thesis_Data//base_gnn_testing_root//processed//data_0.pt")